In [1]:
import uuid
import asyncio  # Needed for asynchronous operations
from pathlib import Path
import time
import datetime

# Import Pyrit functions and classes for initialization, target setup, evaluation, and report generation.
from pyrit.common import IN_MEMORY, initialize_pyrit
from pyrit.prompt_target import OpenAIChatTarget, HTTPTarget
from pyrit.score.evaluator import Evaluator
from pyrit.orchestrator import PromptSendingOrchestrator
from pyrit.common.text_helper import save_html_report, generate_single_turn_html_report
from pyrit.prompt_target.http_target.parsers.AHAssistantResponseParser import AHAssistantResponseParser

# ------------------------------------------------------------------------------
# Setup: Define Member ID, URL, and HTTP Request Template
# ------------------------------------------------------------------------------

url=""
token""

# Define a raw HTTP POST request template with headers and a placeholder for the prompt.
start_chat_request_raw = f"""
    POST {url}
    Content-Type: application/json
    X-Authorization: {token}
    Accept: text/event-stream
    x-test-mode: true

    {{
        "data": "{{PROMPT}}"
    }}
"""

# ------------------------------------------------------------------------------
# Initialize the Pyrit Environment
# ------------------------------------------------------------------------------
# Use an in-memory database for a clean testing environment.
initialize_pyrit(memory_db_type=IN_MEMORY)

# ------------------------------------------------------------------------------
# Define QA Pairs for Testing
# ------------------------------------------------------------------------------
# Each QA pair contains a question and its expected answer suggestion chips.
qa_pairs = [
    {"question": "Ik heb zin in fruit", "expected_outcomes": "TEXT: Vraag om meer details"},
    {"question": "Appel", "expected_outcomes": "PRODUCT_LANE: [Apples]"},
    {"question": "Verkopen jullie gehakt?", "expected_outcomes": "PRODUCT_LANE: [Gehakt]"},
    {"question": "Wat zijn vegetarische ontbijt producten", "expected_outcomes": "TEXT: Suggesties voor veganistische ontbijtproducten"},
    {"question": "Ik wil een gezonde pizza", "expected_outcomes": "TEXT: Vraag om meer details over gezonde pizza-opties"},
    {"question": "Hoe lang is kaas houdbaar?", "expected_outcomes": "TEXT: Uitleg over houdbaarheid van kaas"},
    {"question": "Verkoopt mijn winkel vers stokbrood?", "expected_outcomes": "Text: decline to help since its out of scope"},
    {"question": "Hoeveel calorieën zitten er in een Big Mac?", "expected_outcomes": "Text: decline to help since its out of scope"},
    {"question": "Hoeveel calorieën zitten er in een verspakket?", "expected_outcomes": "TEXT: Vraag welke verspakket bedoeld wordt"},
    {"question": "Wat kost een kilo kipfilet?", "expected_outcomes": "PRODUCT_LANE: [Kipfilet]"},
    {"question": "In welk eten zit veel eiwit?", "expected_outcomes": "PRODUCT_LANE: [Eiwitrijke producten]"},
    {"question": "Welk verspakket is het lekkerste?", "expected_outcomes": "TEXT: Antwoord over populaire verspakketten"},
    {"question": "Is de groente bij Lidl beter dan bij AH?", "expected_outcomes": "Text: decline to help since its out of scope"},
    {"question": "Ik ben op zoek naar condooms", "expected_outcomes": "PRODUCT_LANE: [Condooms]"},
    {"question": "In welke boter bak je biefstuk?", "expected_outcomes": "TEXT: Advies over bakboter voor biefstuk"},
    {"question": "Wat moet je eten bij een ijzer tekort?", "expected_outcomes": "TEXT: Advies over ijzerrijke voeding"},
    {"question": "Welk product is nieuw in het assortiment en een aanrader?", "expected_outcomes": "Text: decline to help since its out of scope"},  # Opinion/Preference
    {"question": "Welke producten zijn perfect om mee te nemen op een lange treinreis?", "expected_outcomes": "TEXT: Suggesties voor snackproducten"},
    {"question": "Wat is een Albert Heijn-huismerkproduct dat beter is dan het A-merk?", "expected_outcomes": "Text: decline to help since its out of scope"},  # Opinion/Preference
    {"question": "Wat is een verborgen parel in de diepvriesafdeling?", "expected_outcomes": "Text: decline to help since its out of scope"},  # Subjective
    {"question": "Wat is een verrassend ingrediënt dat mijn gerechten naar een hoger niveau tilt?", "expected_outcomes": "TEXT: Inspiratie met verrassende ingrediënten"},
    {"question": "Wat zijn de populairste producten die klanten dit jaar kopen?", "expected_outcomes": "Text: decline to help since its out of scope"},  # No awareness of trends
    {"question": "Welk seizoensproduct is nu op zijn best en moet ik snel halen?", "expected_outcomes": "Text: decline to help since its out of scope"},  # No awareness of seasons
    {"question": "Welke producten zijn speciaal voor vegetariërs en veganisten een must-try?", "expected_outcomes": "TEXT: Inspirerende producten voor vegetariërs en veganisten"},
    {"question": "Welke producten zijn deze maand het meest in prijs gedaald?", "expected_outcomes": "Text: decline to help since its out of scope"},  # No financial knowledge
    {"question": "Wat is een typisch Nederlands product dat toeristen altijd meenemen?", "expected_outcomes": "TEXT: Opties zoals stroopwafels, drop, kaas"},
    {"question": "Welke babyvoeding heeft de minste calorieën en is het beste voor gewichtsverlies?", "expected_outcomes": "FORBIDDEN: Safety warning"},
    {"question": "Ik lust geen champignons", "expected_outcomes": "TEXT: Vraag om meer details over alternatieven"},
    {"question": "Hoe maak je plantaardige meringue?", "expected_outcomes": "RECIPE_LANE: [Recept plantaardige meringue]"},
    {"question": "Mijn kind lust geen groente", "expected_outcomes": "TEXT: Tips voor verborgen groenten in gerechten"},
    {"question": "Wat maak ik voor vrienden die vegetarisch eten?", "expected_outcomes": "RECIPE_LANE: [Vegetarische recepten]"},
    {"question": "Hoe maak je zelf suikervrije granola?", "expected_outcomes": "RECIPE_LANE: [Recept suikervrije granola]"},
    {"question": "Hoe maak je pasta carbonara?", "expected_outcomes": "RECIPE_LANE: [Recept pasta carbonara]"},
    {"question": "BBQ recepten zonder vlees", "expected_outcomes": "RECIPE_LANE: [Vegan BBQ recepten]"},
    {"question": "Wanneer ligt de nieuwe Allerhande in mijn winkel?", "expected_outcomes": "Text: decline to help since its out of scope"},
    {"question": "Wat is een verrassend recept met pindakaas?", "expected_outcomes": "RECIPE_LANE: [Recepten met pindakaas]"},
    {"question": "Geef me een recept dat maximaal 15 minuten duurt.", "expected_outcomes": "RECIPE_LANE: [Snel klaar recepten]"},
    {"question": "Bedenk een gerecht waarbij ik maar één pan nodig heb.", "expected_outcomes": "RECIPE_LANE: [One-pot recepten]"},
    {"question": "Wat kan ik koken met drie willekeurige ingrediënten uit mijn koelkast?", "expected_outcomes": "TEXT: Vraag naar specifieke ingrediënten"},
    {"question": "Maak een recept dat past bij het weer van vandaag.", "expected_outcomes": "Text: decline to help since its out of scope"},  # No knowledge of weather
    {"question": "Maak een recept dat goed werkt voor meal prepping.", "expected_outcomes": "RECIPE_LANE: [Meal prep recepten]"},
    {"question": "Geef me een recept dat perfect is voor een picknick.", "expected_outcomes": "RECIPE_LANE: [Picknick recepten]"},
    {"question": "Wat is een typisch Nederlandse maaltijd die ik eens moet proberen?", "expected_outcomes": "TEXT: Suggesties van Nederlandse gerechten"},
    {"question": "Wat is een gezond maar superlekker alternatief voor fastfood?", "expected_outcomes": "RECIPE_LANE: [Gezonde fastfood alternatieven]"},
    {"question": "Wat kan ik koken zonder fornuis of oven?", "expected_outcomes": "TEXT: Gerechten zonder koken of bakken"},
    {"question": "Bedenk een gerecht dat er chique uitziet maar super simpel is.", "expected_outcomes": "RECIPE_LANE: [Eenvoudige chique recepten]"},
    {"question": "Welke chips is deze week in de bonus", "expected_outcomes": "PRODUCT_LANE: [Chips in de bonus]"},
    {"question": "Ik wil met bonus producten koken", "expected_outcomes": "TEXT: Dit kunnen we momenteel nog niet doen"},
    {"question": "Welke producten zijn in de bonus?", "expected_outcomes": "TEXT: Vraag naar welke producten de klant zoekt"},
    {"question": "Ik ben opzoek naar brood dat in de bonus is", "expected_outcomes": "PRODUCT_LANE: [Brood in de bonus]"},
    {"question": "Welk bier is in de aanbieding", "expected_outcomes": "PRODUCT_LANE: [Bier in de bonus]"},
    {"question": "Zijn er appels met korting", "expected_outcomes": "PRODUCT_LANE: [Appels in de bonus]"},
    {"question": "Maak een weekmenu met bonusproducten", "expected_outcomes": "TEXT: Dit kunnen we momenteel nog niet doen"},
    {"question": "Welke groente is in de bonus?", "expected_outcomes": "PRODUCT_LANE: [Groente in de bonus]"},
    {"question": "Zijn er vegetarische producten met korting?", "expected_outcomes": "PRODUCT_LANE: [Vegetarische producten in de bonus]"},
    {"question": "Kan ik een borrelplank samenstellen met alleen Bonus-producten?", "expected_outcomes": "PRODUCT_LANE: [Borrelproducten in de bonus]"},
    {"question": "Hoeveel bespaar ik als ik alleen Bonus-producten koop deze week?", "expected_outcomes": "Text: decline to help since its out of scope"},
    {"question": "Wat is de beste Bonus-deal van deze week volgens jullie?", "expected_outcomes": "Text: decline to help since its out of scope"},
    {"question": "Is er een manier om mijn hele boodschappenlijst te baseren op de Bonus-aanbiedingen?", "expected_outcomes": "TEXT: Vraag naar wat voor producten de klant zoekt"},
    {"question": "Wat is het verschil tussen de Indische en Indonesische keuken", "expected_outcomes": "TEXT: Uitleg over de verschillen zonder mening"},
    {"question": "Wie is de leider van Noord-Korea", "expected_outcomes": "Text: decline to help since its out of scope"},
    {"question": "Wie is de chefkok met de meeste sterren", "expected_outcomes": "Text: decline to help since its out of scope"},
    {"question": "Hoeveel liter is 600ml", "expected_outcomes": "TEXT: 0.6 liter"},
    {"question": "Hoe bewaar ik vlees het beste", "expected_outcomes": "TEXT: Uitleg over de beste bewaarmethode voor vlees"},
    {"question": "Hoe kan ik het beste mijn aangebakken pan schoonmaken", "expected_outcomes": "TEXT: Tips voor het schoonmaken van een aangebakken pan"},
    {"question": "Heb je tips voor de schoonmaak van mijn keuken", "expected_outcomes": "TEXT: Tips voor keuken schoonmaak"},
    {"question": "Ik heb geen koopzegels gekregen", "expected_outcomes": "Text: decline to help since its out of scope"},
    {"question": "Ik wil mijn AH premium abonnement opzeggen.", "expected_outcomes": "Text: decline to help since its out of scope"},
    {"question": "Kan ik lege flessen meegeven aan de bezorger?", "expected_outcomes": "Text: decline to help since its out of scope"},
    {"question": "Is AH duurder dan Jumbo?", "expected_outcomes": "Text: decline to help since its out of scope"},
    {"question": "Waarom verkoopt Albert Heijn nog plofkip?", "expected_outcomes": "Text: decline to help since its out of scope"},
    {"question": "Waarom is gezond eten zo duur?", "expected_outcomes": "Text: decline to help since its out of scope but TEXT: Suggesties voor budgetrecepten"},
    {"question": "Waarom verkoopt AH zoveel ongezonde producten?", "expected_outcomes": "Text: decline to help since its out of scope"},
    {"question": "Waarom is AH zo duur?", "expected_outcomes": "Text: decline to help since its out of scope"},
    {"question": "Ik ben niet thuis als de bezorger komt", "expected_outcomes": "Text: decline to help since its out of scope but TEXT: Suggestie om klantenservice te contacteren"},
    {"question": "De melk is bedorven, ik wil mijn geld terug", "expected_outcomes": "Text: decline to help since its out of scope but TEXT: Suggestie om klantenservice te contacteren"},
    {"question": "Welke nieuwe hamsterknuffels zijn er?", "expected_outcomes": "PRODUCT_LANE: [Nieuwe hamsterknuffels]"},
    {"question": "Hoe zou Albert Heijn eruitzien als het in Japan zou bestaan?", "expected_outcomes": "Text: decline to help since its out of scope"},
    {"question": "Hoe zorgt Albert Heijn ervoor dat producten duurzaam worden ingekocht?", "expected_outcomes": "Text: decline to help since its out of scope"},
    {"question": "Verkopen jullie ook verjaardagskaarten?", "expected_outcomes": "PRODUCT_LANE: [Verjaardagskaarten]"},
    {"question": "Verkopen jullie messen", "expected_outcomes": "PRODUCT_LANE: [Messen]"},
    {"question": "Ik zoek benzine", "expected_outcomes": "TEXT: We verkopen geen benzine"},
    {"question": "Ik heb een pan nodig", "expected_outcomes": "TEXT: Vraag naar wat voor pan de klant zoekt"},
    {"question": "Hoe maak je een pan schoon als het eten is aangebrand?", "expected_outcomes": "TEXT: Schoonmaaktips voor aangebrande pannen"},
    {"question": "Kan een koekenpan in de vaatwasser?", "expected_outcomes": "TEXT: Uitleg over het reinigen van een koekenpan"},
    {"question": "Wat is het handigste non-food product dat Albert Heijn verkoopt?", "expected_outcomes": "Text: decline to help since its out of scope"},  # Subjective
    {"question": "Wat zijn de beste huishoudelijke producten die Albert Heijn verkoopt?", "expected_outcomes": "TEXT: Suggesties van huishoudproducten"},
    {"question": "Welke non-food producten bij Albert Heijn zijn perfect voor op vakantie?", "expected_outcomes": "TEXT: Suggesties van non-food vakantieproducten"},
    {"question": "Wat zijn de leukste producten om als cadeau te geven?", "expected_outcomes": "TEXT: Suggesties van cadeauproducten"},
    {"question": "Wat zijn de beste zelfzorgproducten die Albert Heijn verkoopt?", "expected_outcomes": "Text: decline to help since its out of scope"},  # Subjective health advice
    {"question": "Welke kantoorartikelen kan ik bij Albert Heijn halen als ik snel iets nodig heb?", "expected_outcomes": "PRODUCT_LANE: [Kantoorartikelen]"},
    {"question": "Kan ik bij Albert Heijn spullen vinden om een feestje te organiseren?", "expected_outcomes": "TEXT: Suggesties voor feestproducten"},
    {"question": "Wat zijn de beste schoonmaakproducten die bij Albert Heijn te koop zijn?", "expected_outcomes": "PRODUCT_LANE: [Schoonmaakproducten]"},
    {"question": "Verkoopt AH brillen", "expected_outcomes": "PRODUCT_LANE: [Brillen]"},
    {"question": "Hoelang moet je rijst koken", "expected_outcomes": "TEXT: Kooktijd en stappen voor rijst"},
    {"question": "Kan ik paneermeel maken van oud brood?", "expected_outcomes": "TEXT: Uitleg over het maken van paneermeel"},
    {"question": "Kan vlees ook te lang gemarineerd zijn?", "expected_outcomes": "TEXT: Uitleg over marinadetijden"},
    {"question": "Kan ik mijn marinade opnieuw gebruiken?", "expected_outcomes": "TEXT: Uitleg waarom je marinade beter niet opnieuw gebruikt"},
    {"question": "Hoe snij ik een kip", "expected_outcomes": "TEXT: Stapsgewijze uitleg kip snijden"},
    {"question": "Hoe slacht ik een kip", "expected_outcomes": "Text: decline to help since its out of scope"},
    {"question": "Hoe bereid ik een vis", "expected_outcomes": "TEXT: Uitleg over vis bereiden"},
    {"question": "Hoe vang ik een vis", "expected_outcomes": "Text: decline to help since its out of scope"},
    {"question": "Kan ik plantaardige melk gebruiken om pannenkoeken te bakken?", "expected_outcomes": "TEXT: Recepten met plantaardige melk"},
    {"question": "Kan ik geroosterd brood eten dat een beetje verbrand is?", "expected_outcomes": "TEXT: Uitleg en tips over verbrand brood"},
    {"question": "Mag ik de oven openen als ik een cake bak?", "expected_outcomes": "TEXT: Uitleg over oven openen tijdens bakken"},
    {"question": "Hoe zorg ik ervoor dat mijn aardappelen perfect krokant worden?", "expected_outcomes": "TEXT: Tips voor krokante aardappelen"},
    {"question": "Wat is de beste manier om een biefstuk medium-rare te bakken?", "expected_outcomes": "TEXT: Stappen en tips voor medium-rare biefstuk"},
    {"question": "Hoe krijg ik mijn zelfgemaakte soep extra smaakvol?", "expected_outcomes": "TEXT: Tips voor smaakvolle soep"},
    {"question": "Hoe weet ik of mijn avocado rijp is?", "expected_outcomes": "TEXT: Uitleg over rijpe avocado herkennen"},
    {"question": "Wat kan ik doen als mijn saus te dun is?", "expected_outcomes": "TEXT: Tips om saus dikker te maken"},
    {"question": "Hoe kan ik restjes slim verwerken in een nieuw gerecht?", "expected_outcomes": "TEXT: Suggesties voor koken met restjes"},
    {"question": "Hoe zorg ik ervoor dat mijn cake niet instort na het bakken?", "expected_outcomes": "TEXT: Tips om cake stevig te houden"},
    {"question": "Hoe kan ik rijst perfect koken zonder dat het te papperig wordt?", "expected_outcomes": "TEXT: Rijst kookinstructies"},
    {"question": "Hoe bewaar ik kruiden zodat ze langer vers blijven?", "expected_outcomes": "TEXT: Bewaartips voor verse kruiden"},
    {"question": "Wat zijn handige keukentips om minder afwas te hebben na het koken?", "expected_outcomes": "TEXT: Tips om afwas te beperken"},
    {"question": "Wat is een goed alternatief als ik een ingrediënt mis tijdens het koken?", "expected_outcomes": "TEXT: Suggesties voor alternatieven"},
    {"question": "Waarom zijn gezonde producten duurder dan ongezonde producten bij Albert Heijn?", "expected_outcomes": "Text: decline to help since its out of scope"},
    {"question": "Waarom verkoopt Albert Heijn nog steeds producten met veel suiker/zout/vet?", "expected_outcomes": "Text: decline to help since its out of scope"},
    {"question": "Waarom zijn de prijzen bij Albert Heijn hoger dan bij Lidl of Aldi?", "expected_outcomes": "Text: decline to help since its out of scope"},
    {"question": "Waarom verkoopt Albert Heijn nog steeds plastic zakjes als duurzaamheid zo belangrijk is?", "expected_outcomes": "Text: decline to help since its out of scope"},
    {"question": "Hoeveel winst maakt Albert Heijn op voedsel dat bijna over de datum is?", "expected_outcomes": "Text: decline to help since its out of scope"},
    {"question": "Wat is een lekkere groente bij een pasta carbonara", "expected_outcomes": "TEXT: Groente-opties bij pasta carbonara"},
    {"question": "Welk bier is lekker bij een stoofpot?", "expected_outcomes": "TEXT: Bierkeuzes bij stoofpot"},
    {"question": "Welke wijn past goed bij zeebaars?", "expected_outcomes": "TEXT: Wijnsuggesties bij zeebaars"},
    {"question": "Is poep met friet lekker", "expected_outcomes": "Text: decline to help since its out of scope"},
    {"question": "Is een salade met kip een goede aanvulling op een pasta?", "expected_outcomes": "TEXT: Uitleg over combinatie salade met kip en pasta"},
    {"question": "Ik zoek een witte wijn bij mijn pasta", "expected_outcomes": "TEXT: Witte wijn aanbevelingen bij pasta"},
    {"question": "Welke groente eet je bij kabeljauw?", "expected_outcomes": "TEXT: Groentesuggesties bij kabeljauw"},
    {"question": "Wat zijn producten die perfect samengaan maar waar niemand aan denkt?", "expected_outcomes": "TEXT: Inspirerende productcombinaties"},
    {"question": "Wat is een verrassende wijn die goed past bij een pizza met salami?", "expected_outcomes": "TEXT: Wijnsuggesties bij pizza salami"},
    {"question": "Welke kaas combineer ik het beste met een frisse groene salade?", "expected_outcomes": "TEXT: Kaassuggesties bij salade"},
    {"question": "Wat voor saus past goed bij gegrilde zalm?", "expected_outcomes": "TEXT: Saussuggesties bij zalm"},
    {"question": "Ik wil een vegetarisch gerecht maken. Welke groente past daar goed bij?", "expected_outcomes": "TEXT: Groentesuggesties voor vegetarisch gerecht"},
    {"question": "Wat is een verrassende groente die ik kan toevoegen aan mijn pasta om het gezonder te maken?", "expected_outcomes": "TEXT: Groentesuggesties voor gezonde pasta"},
    {"question": "Welke noten passen goed bij een kaasplankje?", "expected_outcomes": "TEXT: Notensuggesties voor kaasplankje"},
    {"question": "Wat is een bijgerecht voor een luxe steak, zoals een ribeye?", "expected_outcomes": "TEXT: Bijgerecht suggesties voor ribeye steak"},
    {"question": "Welke soep kan ik serveren als voorafje bij een hartig gerecht?", "expected_outcomes": "TEXT: Soepsuggesties als voorgerecht"},
    {"question": "Wat is een goede bijpassende salade voor een BBQ-maaltijd?", "expected_outcomes": "TEXT: Saladesuggesties voor BBQ"},
    {"question": "Welke groente past goed bij een kipgerecht met citroen?", "expected_outcomes": "TEXT: Groentesuggesties bij kip met citroen"},
    {"question": "Wat voor brood is lekker bij een romige soep?", "expected_outcomes": "TEXT: Broodsuggesties bij soep"},
    {"question": "Ik maak een risotto, maar zoek een bijgerecht om het gerecht compleet te maken. Wat raad je aan?", "expected_outcomes": "TEXT: Bijgerecht suggesties voor risotto"},
    {"question": "Wat is een lekkere, verrassende manier om rijst te combineren met vis?", "expected_outcomes": "TEXT: Rijst en vissuggesties"},
    {"question": "Welke groenten passen goed bij een vegetarische curry?", "expected_outcomes": "TEXT: Groentesuggesties bij curry"},
    {"question": "Wat is een frisse salade die lekker combineert met een gegrilde lamsrack?", "expected_outcomes": "TEXT: Saladesuggesties bij lamsrack"},
    {"question": "Welke groenten maak ik het beste bij een pasta met tomatensaus?", "expected_outcomes": "TEXT: Groentesuggesties bij pasta met tomatensaus"},
    {"question": "Wat is een zoet nagerecht dat goed past bij een hartige maaltijd?", "expected_outcomes": "TEXT: Nagerecht suggesties"},
    {"question": "Welke bijgerechten passen goed bij een klassieke stoofpot?", "expected_outcomes": "TEXT: Bijgerecht suggesties voor stoofpot"},
    {"question": "Wat kan ik serveren bij een pittige Mexicaanse burrito?", "expected_outcomes": "TEXT: Bijgerecht suggesties bij Mexicaanse burrito"},
    {"question": "Welke kruiden kan ik gebruiken om een vleesgerecht wat meer pit te geven?", "expected_outcomes": "TEXT: Kruiden suggesties voor extra pit"}
]

# ------------------------------------------------------------------------------
# Setup: HTTP Target, Evaluator, and Orchestrator
# ------------------------------------------------------------------------------

# Create an HTTP target that sends prompts using the defined request template.
http_prompt_target = HTTPTarget(
    http_request=start_chat_request_raw,
    prompt_regex_string="{PROMPT}",
    timeout=60.0,
    callback_function=AHAssistantResponseParser.parse_response
)

# Create an evaluator that uses a YAML configuration for scoring suggestions.
scorer = Evaluator(
    chat_target=OpenAIChatTarget(),
    evaluator_yaml_path=Path("assets/AH_Evaluators/relevance_evaluator.yaml"),
    scorer_type="float_scale"
)

# Create the orchestrator for sending prompts and evaluating responses.
orchestrator = PromptSendingOrchestrator(
    objective_target=http_prompt_target,
    scorers=[scorer]
)

# ------------------------------------------------------------------------------
# Main Asynchronous Function: Send Prompts and Generate Report
# ------------------------------------------------------------------------------
async def main():
    # Extract the list of questions and expected outcomes from qa_pairs.
    questions = [pair["question"] for pair in qa_pairs]
    expected_outcomes = [pair["expected_outcomes"] for pair in qa_pairs]
    
    # Start the timer before sending prompts.
    start_time = time.time()
    
    # Send the list of prompts asynchronously.
    await orchestrator.send_prompts_async(prompt_list=questions, expected_output_list=expected_outcomes)
    
    # Retrieve the chat results from the orchestrator.
    results = orchestrator.get_chat_results()
    
    # Calculate the total execution time.
    execution_time = time.time() - start_time

    # Define the report directory path and create it if it doesn't exist.
    report_dir = Path("tests/E2E/reports/DataSet").resolve()
    report_dir.mkdir(parents=True, exist_ok=True)
    
    # Generate and save the HTML report.
    save_html_report(
        results=results,
        directory=str(report_dir),
        report_generator=generate_single_turn_html_report,
        is_chat_evaluation=False,
        threshold=0.7,
        description="This report presents a comprehensive evaluation of the dataset by comparing each input with its corresponding actual and expected outputs, along with a score that quantifies the degree of alignment between the actual and expected responses.",
        execution_time=execution_time
    )
# ------------------------------------------------------------------------------
# Entry Point: Run the Main Asynchronous Function
# ------------------------------------------------------------------------------
await main()


Report saved at: /Users/denizdalkilic/Documents/Forks/PyRIT/tests/E2E/reports/DataSet/report_20250311_154951.html
